---
## Step Response Tracking
Next, we do the same, but change the setpoint after a few seconds.

In [ ]:
setpoints = np.zeros(EPISODE_LENGTH)
setpoints[:] = 23.0 # °C
setpoints[int(EPISODE_LENGTH/2):] = 40.0 # °C

plant_control = PlantControl(IS_HARDWARE, SAMPLE_RATE)
stepped_setpoint_results = plant_control.episode(setpoints, (50.0, 0.001, 0.1))

stepped_setpoint_results.sample(10).sort_values(COL_TIME)


In [ ]:
plot_episode(stepped_setpoint_results)
